<a href="https://colab.research.google.com/github/Feliz-ua/goit-math-hw-03/blob/main/hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Завдання 1. Аналіз динаміки користувацької активності**

**Аналітичний розв'язок**

Маємо функцію, яка описує дінамику активних сесій на платформі:
$$ f(t)=1000⋅t⋅e^{-0.2t}
$$
де $t$ -час у годинах від початку робочого дня (08:00),
а $f(t)$ - кількість одночасно активних сесій

- - -
Рішення:
1. Виділяємо множники\
Запишемо функцію у вигляді добутку:
$$f(t)=u(t)\cdot v(t)$$
де
$$ u(t)=1000⋅t $$
$$ v(t)=e^{-0.2t}$$

2. Обчислюємо похідні для кожного множника ($u'$ та $v'$):
$$u'(t)=(1000t)'=1000$$
$$v'(t)=(-0.2)\cdot e^{-0.2t}=-0.2e^{-0.2t}$$

3. Застосовуємо правило добутку:\
Формула правила добутку:
$$(u \cdot v)'=u'⋅v+u⋅v'$$
Підставляємо знайдені значення:
$$f'(t)=1000e^{-0.2t}+1000t\cdot(-0.2e^{-0.2t})$$

$$f'(t)=1000⋅e^{-02t}(1-0.2t)$$
4. Знайдемо момент пикового навантаження:\
Пікове навантаження досягається, коли швидкість зміни дорівнює нулю ($f'(t)=0$).\
Оскільки $e^{-0.2t}$ ніколи не дорівнює 0, прирівнюємо до нуля вираз у дужках:
$$1-0.2t=0 \\t=1/0.2=5$$
Робочий день починається о 08:00, тому $t=5$ означає 08:00 +5 годин =13:00.
Отже, пікове навантаження на платформу припадає на 13:00.

- - -



In [ ]:
import numpy as np
from scipy.optimize import approx_fprime

# 1. Визначення функцій
def f_load(x):
    """
    Функція навантаження f(t).
    approx_fprime очікує масив, тому беремо t = x[0]
    """
    t = x[0]
    return 1000 * t * np.exp(-0.2 * t)

def f_derivative_analytical(t):
    """Аналітична формула похідної f'(t)"""
    return 1000 * np.exp(-0.2 * t) * (1 - 0.2 * t)

# 2. Параметри для аналізу
time_points = [2, 6, 10]  # t=2 (10:00), t=6 (14:00), t=10 (18:00)
epsilon = np.sqrt(np.finfo(float).eps) # Оптимальний крок для чисельного методу

print(f"{'Час (t)':<10} {'Година':<10} {'Чисельна f\'(t)':<20} {'Аналітична f\'(t)':<20} {'Різниця'}")
print("-" * 75)

# 3. Обчислення та виведення
for t in time_points:
    # Чисельне диференціювання
    # approx_fprime повертає масив градієнтів, беремо [0]
    f_prime_num = approx_fprime(np.array([t]), f_load, epsilon)[0]

    # Аналітичне обчислення
    f_prime_ana = f_derivative_analytical(t)

    # Форматування часу
    clock_time = f"{8+t}:00"

    # Різниця
    diff = abs(f_prime_num - f_prime_ana)

    print(f"{t:<10} {clock_time:<10} {f_prime_num:<20.4f} {f_prime_ana:<20.4f} {diff:.2e}")

- - -
**Завдання 2. Моделювання процесу навчання** \
Потрібно змоделювати, як студент засвоює новий матеріал з часом. Педагогічні дослідження показують, що швидкість навчання пропорційна кількості ще не засвоєного матеріалу.\
Цю залежність описує наступне диференціальне рівняння:
$$\frac{\mathrm{d} K}{\mathrm{d} t}=r(M-K)$$
де:
*   $K(t)$-рівень знань студента в момент часу $t$ (у відсотках, від 0 до 100)
*   $M=100$ - максимальний ріень знань (100%)
*   $r=0.15$ - коефіцієнт швидкості навчання (1/день)
*   Початкова умова $K(0)=10 (студент починає з 10% базових знань)
- - -


In [ ]:
# Завдання 2. Моделювання процесу навчання (чисельний метод рішення)
import numpy as np
from scipy.integrate import solve_ivp

M = 100
r = 0.15
TARGET = 90

def learning_rate(t, K):
    return r * (M - K)

def hit_90(t, K):
    return K[0] - TARGET
hit_90.terminal = True
hit_90.direction = 1

def time_to_90(K0):
    sol = solve_ivp(
        learning_rate,
        (0, 30),
        y0=[K0],
        events=hit_90
    )
    return None if sol.t_events[0].size == 0 else float(sol.t_events[0][0])

# Дані
K0_list = [5, 10, 20]
rows = [(K0, time_to_90(K0)) for K0 in K0_list]

# База: K(0)=5
t90_base = rows[0][1]

# Таблиця
header = "{:<8} | {:>12} | {:>24}".format("K(0), %", "t_90, днів", "Δt = t90(K0)-t90(5), дн")
line = "-" * len(header)

print(header)
print(line)

for i, (K0, t90) in enumerate(rows):
    # t_90
    t90_str = "—" if t90 is None else "{:.4f}".format(t90)

    # Δt: для базового рядка пусто; для інших — від’ємні значення, якщо швидше
    if i == 0 or t90 is None or t90_base is None:
        diff_str = "—"
    else:
        diff = t90 - t90_base   # буде < 0, якщо K0 досягає 90% швидше за K0=5
        diff_str = "{:.4f}".format(diff)

    print("{:<8} | {:>12} | {:>24}".format(K0, t90_str, diff_str))

---
**Висновки:** \
На основі розрахунків можна зробити наступний висновок. \
Час досягнення 90% рівня знань залежить від початкової підготовки:
*   При $К(0)=5$ необхідний час складає приблизно 15,0 днів.
*   При $К(0)=10$ необхідний час складає приблизно 14,6 днів.
*   При $К(0)=15$ необхідний час складає приблизно 13,9 днів.

Звідси, чим вищий початковий рівень знань студента, тим менше часу потрібно йому для досягнення рівня у 90%. Різниця між стартом з 5% та 20% становить понад 1 день.


